In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import re

import json 
import torch
import json_repair
import pandas as pd
from glob import glob 
from openai import OpenAI
from pydantic import BaseModel
from dotenv import load_dotenv
from pqdm.processes import pqdm
from datasets import Dataset, load_dataset
from trl import SFTTrainer
from peft import AutoPeftModelForCausalLM, LoraConfig
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          TrainingArguments, 
                          pipeline)


# 제작한 데이터세 불러오기 
file_list = glob("./data/*.csv")
print(file_list)

df = pd.concat([pd.read_csv(file) for file in file_list])
df.shape

/home/admin/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['./data/중고나라_게시물_final_20250313_124859.csv', './data/쇼핑 고객 서비스_final_20250313_111949.csv', './data/은행 상담_final_20250313_105525.csv', './data/강의_세미나_홍보글_final_20250313_124859.csv', './data/이벤트_안내글_final_20250313_124859.csv', './data/보험 상담_final_20250313_111949.csv', './data/제품_리뷰글_final_20250313_124859.csv', './data/모임_동호회_홍보글_final_20250313_124859.csv']


(551, 8)

In [3]:
df['messages'] = df.apply(
    lambda row: [
        {"role": "user", "content": row['origin_data']},
        {"role": "assistant", "content": row['anonymized_data']}
    ],
    axis=1
)

import datasets 

dataset = datasets.Dataset.from_pandas(df)

def get_chat_format(element):
    system_prompt = "너는 개인정보를 비식별화하는 Assistant야. 너는 주어진 데이터를 바탕으로 개인정보를 비식별화하는 작업을 해야해."

    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": element["origin_data"]},
            {"role": "assistant", "content": element["anonymized_data"]},
        ]
    }

dataset = dataset.map(get_chat_format, remove_columns=dataset.features, batched=False)
dataset = dataset.train_test_split(test_size=0.1, seed=42)


Map: 100%|██████████| 551/551 [00:00<00:00, 15850.58 examples/s]


In [20]:
for idx in range(len(dataset["test"])):
    print(dataset["test"][1]["messages"][1]["content"])
    print(dataset["test"][1]["messages"][2]["content"])
    break 


🎉[2023 소외된 창작자들을 위한 글로벌 예술 워크숍]에 여러분을 초대합니다!🎨

안녕하세요, 예술을 사랑하는 여러분! 일상의 바쁜 흐름 속에서도 예술에 대한 열정을 잃지 않는 모든 분들을 위한 특별한 시간을 준비했습니다. 이번 워크숍은 소외된 창작자들이 국제적으로 연결되고 협력할 수 있는 기회를 제공하는 행사입니다.

🌟 모임명: 2023 글로벌 예술 워크숍 - '창작의 새로운 물결'
📅 일시: 2023년 11월 22일(수) 오후 2시 - 6시
📍 장소: 서울시 종로구 인사동길 15, 문화예술센터 3층 대강당

이번 워크숍에서는 글로벌 아티스트들과의 만남뿐만 아니라, 다양한 창작 기법을 배우고 직접 체험할 수 있는 시간도 마련되어 있습니다. 🎶🎨

참가를 원하는 분들은 다음 정보를 통해 신청해 주시기 바랍니다. 참가비는 무료이며, 선착순으로 마감됩니다.

📧 가입 방법: 이메일 artworkshop2023@gmail.com으로 신청서 발송
☎️ 문의 전화: 010-1234-5678 (담당자: 김예술 실장)
📲 카카오톡: ID - artlove2023

여러분의 많은 관심과 참여 부탁드립니다! ✨ 여러분과 함께 새로운 예술의 장을 펼쳐 나갈 수 있기를 기대합니다.😊

*주차가 어려우니 대중교통을 이용해주세요.* 🚇
🎉[2023 소외된 창작자들을 위한 글로벌 예술 워크숍]에 여러분을 초대합니다!🎨

안녕하세요, 예술을 사랑하는 여러분! 일상의 바쁜 흐름 속에서도 예술에 대한 열정을 잃지 않는 모든 분들을 위한 특별한 시간을 준비했습니다. 이번 워크숍은 소외된 창작자들이 국제적으로 연결되고 협력할 수 있는 기회를 제공하는 행사입니다.

🌟 모임명: 2023 글로벌 예술 워크숍 - '창작의 새로운 물결'
📅 일시: 2023년 11월 22일(수) 오후 2시 - 6시
📍 장소: [LOCATION1], 문화예술센터 3층 대강당

이번 워크숍에서는 글로벌 아티스트들과의 만남뿐만 아니라, 다양한 창작 기법을 배우고 직접 체험할 수 있는 시간도 마련되어 있습니다. 🎶🎨

참

'안녕하세요! 😊 여러분의 바이오 열정과 꿈을 지원하기 위해 특별한 모임을 준비했습니다. \n\n모임명: **바이오 혁신 포럼: K바이오의 미래**\n\n활동 내용: 이번 포럼에서는 바이오산업의 현재와 미래를 함께 논의하고, 최신 바이오 기술 동향을 공유합니다. 특별히 [PERSON1] 회장과 [PERSON2] 부회장이 직접 참석하여 K바이오 시대의 비전을 제시할 예정입니다. 혁신적인 스타트업과 업계 전문가들을 만나 네트워킹할 수 있는 기회도 놓치지 마세요!\n\n모임 일시: 2023년 12월 10일 (일) 오후 2시\n\n장소: [LOCATION1], [LOCATION2] 8층 컨퍼런스홀\n\n가입 방법: 포럼 참가 신청은 이메일로 받습니다. [EMAIL1]으로 성함, 연락처, 소속을 적어 신청해주세요. 선착순 100명 한정이니 서둘러 주세요!\n\n연락처: \n- 이메일: [EMAIL1]\n- 전화번호: [CONTACT1]\n- 카카오톡 ID: [KAKAO_ID1]\n\n풍부한 경험과 열정을 가진 여러분의 많은 참여 기대합니다! 🤝 여러분의 참여가 K바이오의 미래를 밝힐 수 있는 중요한 발걸음이 될 것입니다. 궁금한 점이 있다면 언제든지 연락 주시기 바랍니다. ✨\n\n감사합니다!'

In [ ]:
# 학습한 모델을 경로를 지정합니다.
peft_model_id = "./model_Lora"


# PEFT 어댑터를 통해 사전 학습된 모델을 로드합니다.
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)

# 토크나이저 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
tokenizer.padding_side = 'right'  
tokenizer.pad_token = tokenizer.eos_token


In [19]:
def extract_placeholder_mapping(original_text, transformed_text, allowed_types):
    allowed_pattern = re.compile(r'\[(' + '|'.join(allowed_types) + r')\d*\]')
    generic_pattern = re.compile(r'(\[[^]]+\])')

    mapping = {}

    orig_lines = original_text.splitlines()
    trans_lines = transformed_text.splitlines()
    n_lines = min(len(orig_lines), len(trans_lines))

    for idx in range(n_lines):
        orig_line = orig_lines[idx]
        trans_line = trans_lines[idx]

        parts = re.split(generic_pattern, trans_line)
        orig_pos = 0

        for i, part in enumerate(parts):
            if allowed_pattern.match(part):
                # placeholder 발견
                # 다음 literal을 찾음
                next_literal = parts[i + 1] if i + 1 < len(parts) else ''
                
                # 다음 literal이 존재하면, 그 literal까지의 텍스트를 추출
                if next_literal:
                    next_idx = orig_line.find(next_literal, orig_pos)
                    if next_idx != -1:
                        replaced_text = orig_line[orig_pos:next_idx]
                        orig_pos = next_idx
                    else:
                        # 다음 literal을 못 찾으면 끝까지
                        replaced_text = orig_line[orig_pos:]
                        orig_pos = len(orig_line)
                else:
                    # 다음 literal이 없으면 남은 텍스트 전체
                    replaced_text = orig_line[orig_pos:]
                    orig_pos = len(orig_line)

                replaced_text = replaced_text.strip()
                if replaced_text:
                    mapping[replaced_text] = part

            else:
                # literal인 경우, 원본에서 위치 업데이트
                found_idx = orig_line.find(part, orig_pos)
                if found_idx != -1:
                    orig_pos = found_idx + len(part)

    return mapping


extract_placeholder_mapping(
    dataset["test"][1]["messages"][1]["content"], 
    dataset["test"][2]["messages"][2]["content"], 
    allowed_types=(
        "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", 
        "EMAIL", "LOCATION", "KAKO_ID", "TIWTTER_ID", "TELEGRAM_ID")), 


({'🎉[2023 소외된 창작자들을 위한 글로벌 예술 워크숍]에 여러분을 초대합니다!🎨': '[PERSON1]',
  '안녕하세요, 예술을 사랑하는 여러분! 일상의 바쁜 흐름 속에서도 예술에 대한 열정을 잃지 않는 모든 분들을 위한 특별한 시간을 준비했습니다. 이번 워크숍은 소외된 창작자들이 국제적으로 연결되고 협력할 수 있는 기회를 제공하는 행사입니다.': '[PERSON2]',
  "🌟 모임명: 2023 글로벌 예술 워크숍 - '창작의 새로운 물결'": '[PERSON1]',
  '📍 장소: 서울시 종로구 인사동길 15, 문화예술센터 3층 대강당': '[EMAIL1]',
  '이번 워크숍에서는 글로벌 아티스트들과의 만남뿐만 아니라, 다양한 창작 기법을 배우고 직접 체험할 수 있는 시간도 마련되어 있습니다. 🎶🎨': '[PERSON1]',
  '참가를 원하는 분들은 다음 정보를 통해 신청해 주시기 바랍니다. 참가비는 무료이며, 선착순으로 마감됩니다.': '[ADDRESS1]',
  '*주차가 어려우니 대중교통을 이용해주세요.* 🚇': '[PERSON2]'},)